In [1]:
import grakel
from grakel.kernels import WeisfeilerLehman, VertexHistogram, ShortestPath, SubgraphMatching
import json
import os
from rdflib import Graph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
import time
from tqdm import tqdm

In [2]:
INTPUT_DIR = '/home/pieter/Documents/ugent/phd/projects/PreDICT/PROTEGO/datasets/data/output/patients'
BASE_DIR = 'benchmarks/DASHB'

PATIENT_TEMPLATE = 'http://example.com/tx/patients/{pid}'

G_nx = []
patients = []
for graph_f in os.listdir(INTPUT_DIR):
    if graph_f.endswith('.ttl'):
        graph = Graph()
        graph.parse(os.path.join(INTPUT_DIR, graph_f), format='turtle')
        
        networkx_graph = rdflib_to_networkx_digraph(graph, edge_attrs=lambda s, p, o: {'label': p})
        # Add node labels
        for id_, data in networkx_graph.nodes(data=True):
            data['label'] = id_
            
        G_nx.append(networkx_graph)
        patients.append(PATIENT_TEMPLATE.format(pid=graph_f.split('.')[0]))
        
graphs = list(grakel.utils.graph_from_networkx(G_nx, node_labels_tag='label', edge_labels_tag='label', edge_weight_tag='weight'))

In [3]:
# gk = ShortestPath(normalize=True)

# gk.fit_transform(graphs[:1])

In [4]:
# similarities = gk.transform(graphs[1:])

In [5]:
# similarities_ = {}
# for idx, sim in enumerate(similarities):
#     similarities_[patients[idx + 1]] = sim

# sorted_similarities = sorted(similarities_.items(), key=lambda x: x[1], reverse=True)
# print(sorted_similarities[:10])

In [6]:
# gk = SubgraphMatching(with_labels=False, normalize=True)

# gk.fit_transform(graphs[:1])

In [7]:
# simularities = gk.transform(graphs[1:])

In [8]:
# https://ysig.github.io/GraKeL/0.1a8/auto_examples/document_retrieval_example.html

# Initialize Weisfeiler-Lehman subtree kernel
# gk = WeisfeilerLehman(n_iter=2, normalize=True, base_graph_kernel=VertexHistogram)

# print("Computing similarities\n")
# t0 = time.time()

# gk.fit(graphs[:1])
# similarities = gk.transform(graphs[1:])

# print("done in %0.3fs\n" % (time.time() - t0))

In [9]:
# similarities_ = {}
# for idx, sim in enumerate(similarities):
#     similarities_[patients[idx + 1]] = sim

# sorted_similarities = sorted(similarities_.items(), key=lambda x: x[1], reverse=True)
# print(sorted_similarities[:10])

In [10]:
# Run for all patients
similarities = {}
for idx, pid in enumerate(tqdm(patients)):
    gk = WeisfeilerLehman(n_iter=2, normalize=True, base_graph_kernel=VertexHistogram)
    check = gk.fit_transform([graphs[idx]])[0]
    
    assert (check == 1)
    
    similarities[pid] = {}
    for idx_, pid_ in enumerate(patients):
        if pid == pid_:
            similarities[pid][pid_] = float(check)
        if pid_ in similarities.keys():
            similarities[pid][pid_] = similarities[pid_][pid]
        else:
            similarities[pid][pid_] = float(gk.transform([graphs[idx_]])[0])

100%|█████████████████████████████████████████| 282/282 [01:22<00:00,  3.40it/s]


In [11]:
with open(os.path.join(BASE_DIR, 'results', 'WeisfeilerLehman.json'), 'w') as output_f:
    output_f.write(json.dumps(similarities))